In [ ]:
import numpy as np 
import pandas as pd 

# **Reading Datasets**

In [ ]:
data1=pd.read_csv('/kaggle/input/dataset/data_csv.csv')
data2=pd.read_csv('/kaggle/input/autism-screening-for-toddlers/Toddler Autism dataset July 2018.csv')
data3=pd.read_csv('/kaggle/input/autism-screening-on-adults/autism_screening.csv')

In [ ]:
data1.head()

In [ ]:
data2.head()

In [ ]:
data3.head()

In [ ]:
print(data1.columns, data1.shape)

print(data2.columns, data2.shape)
print(data3.columns, data3.shape)


In [ ]:
# print(np.array(data1.columns))
common_elements_1_2 = np.intersect1d(np.array(data1.columns), np.array(data2.columns))
common_elements_1_2_3 = np.intersect1d(common_elements_1_2, np.array(data1.columns))
print((common_elements_1_2_3))

# **Creating dataframes with similar columns and features**

In [ ]:
df1=pd.concat([data1.iloc[:,1:11],data1.iloc[:,[12,22,23,24,25,26,27]]],axis=1)
print(df1.shape)
df1.head()

In [ ]:
df2=pd.concat([data2.iloc[:,1:12],data2.iloc[:,13:]],axis=1)
df2['Age_Mons']=(df2['Age_Mons']/12).astype(int)
print(df2.shape)
df2.head()

In [ ]:
df3=pd.concat([data3.iloc[:,0:15],data3.iloc[:,-2:]],axis=1)
print(df3.shape)
df3.head()

In [ ]:
order_test= pd.DataFrame({
    'df1': df1.columns,
    'df2': df2.columns ,
    'df3': df3.columns
})
order_test

**All columns are in order, now we just join dfs on axis 0**

In [ ]:
# Rename columns to have the same names in all DataFrames
df2.columns = df3.columns = df1.columns

# Concatenate the DataFrames
data_fin = pd.concat([df3, df2, df1], axis=0)
data_fin.head()

In [ ]:
data_fin.shape

In [ ]:
# Get object type columns
object_cols = data_fin.select_dtypes('O').columns

# Create new DataFrame
object_df = pd.DataFrame({
    'Objects': object_cols,
    'Unique values': [data_fin[col].unique() for col in object_cols],
    'number of unique values':[data_fin[col].nunique()for col in object_cols]
})

object_df

In [ ]:
data_fin.columns

In [ ]:
for col in ['Sex', 'Ethnicity',
       'Jaundice', 'Family_mem_with_ASD', 'Who_completed_the_test',
       'ASD_traits']:
    print("-------------------------------")
    print(f'Column name: {col}\n')
    print(f'Unique values:\n{data_fin[col].unique()}')

In [ ]:
replacements = {
    'f': 'F',
    'm': 'M',
}
data_fin['Sex'] = data_fin['Sex'].replace(replacements)

In [ ]:
replacements = {
    'yes': 'Yes',
    'no': 'No',
}
data_fin['Jaundice'] = data_fin['Jaundice'].replace(replacements)

In [ ]:
replacements = {
    'yes': 'Yes',
    'no': 'No',
}
data_fin['Family_mem_with_ASD'] = data_fin['Family_mem_with_ASD'].replace(replacements)

In [ ]:
replacements = {
    'YES': 'Yes',
    'NO': 'No',
}
data_fin['ASD_traits'] = data_fin['ASD_traits'].replace(replacements)

In [ ]:
replacements = {
    'middle eastern': 'Middle Eastern',
    'Middle Eastern ': 'Middle Eastern',
    'mixed': 'Mixed',
    'asian': 'Asian',
    'black': 'Black',
    'south asian': 'South Asian',
    'PaciFica':'Pacifica',
    'Pasifika':'Pacifica'
    
}
data_fin['Ethnicity'] = data_fin['Ethnicity'].replace(replacements)

In [ ]:
replacements = {
    'Health care professional':'Health Care Professional',
    'family member':'Family Member',
    'Family member':'Family Member'
}
data_fin['Who_completed_the_test'] = data_fin['Who_completed_the_test'].replace(replacements)

In [ ]:
for col in ['Sex', 'Ethnicity',
       'Jaundice', 'Family_mem_with_ASD', 'Who_completed_the_test',
       'ASD_traits']:
    print("-------------------------------")
    print(f'Column name: {col}\n')
    print(f'Unique values:\n{data_fin[col].unique()}')

In [ ]:
data_fin['Ethnicity'].replace('?', np.nan, inplace=True)
data_fin['Who_completed_the_test'].replace('?', np.nan, inplace=True)

In [ ]:
pd.DataFrame(data_fin.isnull().sum(), 
             columns=["Missing Values"]).style.bar(color = "#84A9AC")

In [ ]:
idf=data_fin.copy()
from sklearn.impute import SimpleImputer

In [ ]:
imp = SimpleImputer(strategy='most_frequent')
imputed_data = pd.DataFrame(imp.fit_transform(idf))
imputed_data.columns = idf.columns
imputed_data.index = idf.index

In [ ]:
pd.DataFrame(imputed_data.isnull().sum(), 
             columns=["Missing Values"]).style.bar(color = "#84A9AC")

# **Now I wanna filter the final dataset based on age groups**
age groups will be: 
* 0-10 children
* 11-15 adolescent
* 16-24 young
* more than 25 adults

In [ ]:
imputed_data.shape

In [ ]:
imputed_data

In [ ]:
data = imputed_data.copy()

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

In [ ]:
lr = LinearRegression()
dtc = DecisionTreeClassifier()
gclf1 =GaussianNB()
mclf2 = MultinomialNB()
bclf3 =  BernoulliNB()
knn = KNeighborsClassifier()
lgr = LogisticRegression()
rfc = RandomForestClassifier(max_depth = 10, random_state=0)

In [ ]:
data = pd.get_dummies(data, columns= ['Ethnicity', 'Who_completed_the_test'], drop_first =  True)
data.head()

In [ ]:
data['Sex'].replace({"M":1, "F":0}, inplace = True)
data['Jaundice'].replace({"Yes":1, "No":0}, inplace = True)
data['Family_mem_with_ASD'].replace({"Yes":1, "No":0}, inplace = True)
# data['ASD_traits'].replace({"Yes":1, "No":0}, inplace = True)
data.head()

In [ ]:
y = data['ASD_traits']
x = data.drop(columns = ['ASD_traits'])

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(x, y, random_state=12, test_size=0.2)

In [ ]:
dtc.fit(X_train, Y_train)
y_pred_dtc = dtc.predict(X_test)

In [ ]:
# NB
gclf1.fit(X_train, Y_train)
mclf2.fit(X_train, Y_train)
bclf3.fit(X_train, Y_train)

y_pred_gclf1=  gclf1.predict(X_test)
y_pred_mclf2=  mclf2.predict(X_test)
y_pred_bclf3=  bclf3.predict(X_test)

In [ ]:
#knn
knn.fit(X_train, Y_train)
y_pred_knn= knn.predict(X_test)

In [ ]:
#logistic regression
lgr.fit(X_train,Y_train)
Y_pred_lgr = lgr.predict(X_test)

In [ ]:
# Random Forest
rfc.fit(X_train, Y_train)
Y_pred_rfc = rfc.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

In [ ]:
print("Decision Tree Classifier: ",accuracy_score(Y_test, y_pred_dtc))
print("GuassianNB: ",accuracy_score(Y_test, y_pred_gclf1))
print("MultinomialNB: ",accuracy_score(Y_test, y_pred_mclf2))
print("BernoulliNB: ",accuracy_score(Y_test, y_pred_bclf3))
print("KNN Classifier: ",accuracy_score(Y_test, y_pred_knn))
print("Logistic Regression: ",accuracy_score(Y_test, Y_pred_lgr))    
print("Random Forest Classifier: ",accuracy_score(Y_test, Y_pred_rfc))   
print("Confusion matrix of RFC: "), confusion_matrix(Y_test, Y_pred_rfc)

In [ ]:
from sklearn.ensemble import VotingClassifier
from sklearn.svm import SVC

In [ ]:
classifier3 = SVC(probability=True)
voting_classifier = VotingClassifier(estimators=[
('dtc', dtc),
# ('gclf1', gclf1),
# ('mclf2', mclf2),
# ('bclf3', bclf3),
('knn', knn),
('lgr', lgr),
('rfc', rfc),
('svm', classifier3)
], voting='soft')
voting_classifier.fit(X_train, Y_train)
y_pred = voting_classifier.predict(X_test)

# Calculate the accuracy of the ensemble
accuracy = accuracy_score(Y_test, y_pred)
print("Ensemble Accuracy:", accuracy)

In [ ]:
import tensorflow
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_trained_scaled= scaler.fit_transform(X_train)
X_test_scaled= scaler.transform(X_test)
X_trained_scaled[0]


In [ ]:
print(Y_train.unique())

In [ ]:
X_trained_scaled
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
Y_train_encoded= label_encoder.fit_transform(Y_train)
Y_test_encoded= label_encoder.fit_transform(Y_test)

Y_train_encoded, Y_test_encoded

In [ ]:
model = Sequential()

# model.add(Flatten(input_shape=(28,28)))
model.add(Dense(128,activation='relu', input_shape=(X_trained_scaled.shape[1],) ))
model.add(Dense(128,activation='relu'))
model.add(Dense(2,activation='sigmoid'))
model.summary()

In [ ]:
model.compile(loss='sparse_categorical_crossentropy',optimizer='Adam',metrics=['accuracy'])
history = model.fit(X_trained_scaled,Y_train_encoded,epochs=50)
y_prob = model.predict(X_test_scaled)

In [ ]:
y_pred2 = y_prob.argmax(axis=1)
print(y_pred2,"\n", Y_test_encoded, Y_test_encoded.shape)

In [ ]:
print("ConfusionMatrix of ANN: : ",confusion_matrix(Y_test_encoded, y_pred2))
print("Accuracy Score of ANN: ",accuracy_score(Y_test_encoded, y_pred2))

In [ ]:
df_child=imputed_data[imputed_data['Age_Years']<=10].reset_index().drop("index", axis = 1)
df_child

In [ ]:
data_child = df_child.copy()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
# for i in data_child['Who_completed_the_test'].unique():
# #     print(i)
#     print(f"For :{i} ", data_child[i].value_counts())
# data_child['Who_completed_the_test'].describe()
# data_child['Who_completed_the_test'].unique()

data_child['Ethnicity'].value_counts()

In [ ]:
data_child["ASD_traits"].value_counts()

In [ ]:
plt.figure(figsize = (16,7))
sns.countplot(x = 'Who_completed_the_test', data = data_child)
plt.show()

In [ ]:
np.size(np.array(data_child.columns)), data_child.columns, data_child.ASD_traits.describe()

In [ ]:
data_child = pd.get_dummies(data_child, columns= ['Ethnicity', 'Who_completed_the_test'], drop_first =  True)
data_child.head()


In [ ]:
data_child['Sex'].replace({"M":1, "F":0}, inplace = True)
data_child['Jaundice'].replace({"Yes":1, "No":0}, inplace = True)
data_child['Family_mem_with_ASD'].replace({"Yes":1, "No":0}, inplace = True)
# data['ASD_traits'].replace({"Yes":1, "No":0}, inplace = True)
data_child.head()
data_child.shape

In [ ]:
y_child = data['ASD_traits']
x_child = data.drop(columns = ['ASD_traits'])

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(x_child, y_child, random_state=12, test_size=0.2)

In [ ]:
Y_test_net = Y_test.replace({"Yes":1, "No":0})
Y_train_net = Y_train.replace({"Yes":1, "No":0})

Y_test_net

In [ ]:
dtc.fit(X_train, Y_train)
# y_score_dtc = dtc.predict_proba(X_test)[:,1]
y_pred_dtc = dtc.predict(X_test)


# NB
gclf1.fit(X_train, Y_train)
mclf2.fit(X_train, Y_train)
bclf3.fit(X_train, Y_train)

y_pred_gclf1=  gclf1.predict(X_test)
y_pred_mclf2=  mclf2.predict(X_test)
y_pred_bclf3=  bclf3.predict(X_test)

#knn
knn.fit(X_train, Y_train)
y_pred_knn= knn.predict(X_test)
y_score_knn = knn.predict_proba(X_test)[:,1]

#logistic regression
lgr.fit(X_train,Y_train)
Y_pred_lgr = lgr.predict(X_test)
y_score_lgr = lgr.predict_proba(X_test)[:,1]
# y_score_ = dtc.predict_proba(X_test)

# Random Forest
rfc.fit(X_train, Y_train)
Y_pred_rfc = rfc.predict(X_test)
y_score_rfc = rfc.predict_proba(X_test)



In [ ]:
from sklearn.metrics import roc_curve

In [ ]:
y_score_lgr

In [ ]:
fpr_lgr, tpr_lgr, thresholds_lgr = roc_curve(Y_test_net, y_score_lgr)
# fpr, tpr, thresholds = roc_curve(Y_test_net, y_score_dtc)  

In [ ]:
thresholds_lgr

In [ ]:
import plotly.graph_objects as go
trace0 = go.Scatter(
    x = fpr_lgr,
    y = tpr_lgr,
    mode = 'lines',
    name = 'ROC curve'
)

n = 10
indices = np.arange(len(thresholds_lgr))%n == 0

trace1 = go.Scatter(
    x = fpr_lgr[indices],
    y = tpr_lgr[indices],
    mode = 'markers+text',
    name = 'Threshold Points',
    text = [f"Thr={thr:.2f}" for thr in thresholds_lgr[indices] ],
    textposition = 'top center'
    
)

trace2 = go.Scatter(
    x = [0,1],
    y = [0,1],
    mode = 'lines',
    name = 'Random (Area = 0.5)',
    line = dict(dash= 'dash')
    
)

data = [trace0, trace1, trace2]

layout = go.Layout(
    title='Receiver Operating Characteristic',
    xaxis=dict(title='False Positive Rate'),
    yaxis=dict(title='True Positive Rate'),
    autosize=False,
    width=800,
    height=800,
    showlegend=False
)

# Define figure and add data
fig = go.Figure(data=data, layout=layout)

# Show figure
fig.show()

In [ ]:
from sklearn.metrics import precision_recall_fscore_support

In [ ]:
print("Decision Tree Classifier: ",accuracy_score(Y_test, y_pred_dtc))
print("GuassianNB: ",accuracy_score(Y_test, y_pred_gclf1))
print("MultinomialNB: ",accuracy_score(Y_test, y_pred_mclf2))
print("BernoulliNB: ",accuracy_score(Y_test, y_pred_bclf3))
print("KNN Classifier: ",accuracy_score(Y_test, y_pred_knn))
print("Logistic Regression: ",accuracy_score(Y_test, Y_pred_lgr))    
print("Random Forest Classifier: ",accuracy_score(Y_test, Y_pred_rfc))    
print("CMatrix for rfc: \n", confusion_matrix(Y_test, Y_pred_rfc))

In [ ]:
precision, recall, f1_score, support = precision_recall_fscore_support(Y_test, Y_pred_rfc)
print("Precision is : ", precision)
print("Recall is : ", recall)
print("f1 score is : ", f1_score)

In [ ]:
confusion_matrix(Y_test, Y_pred_rfc)

In [ ]:
classifier3 = SVC(probability=True)
voting_classifier = VotingClassifier(estimators=[
('dtc', dtc),
('gclf1', gclf1),
('mclf2', mclf2),
('bclf3', bclf3),
('knn', knn),
('lgr', lgr),
('rfc', rfc),
('svm', classifier3)
], voting='soft')
voting_classifier.fit(X_train, Y_train)
y_pred = voting_classifier.predict(X_test)

# Calculate the accuracy of the ensemble
accuracy = accuracy_score(Y_test, y_pred)
print("Ensemble Accuracy:", accuracy)

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_trained_scaled= scaler.fit_transform(X_train)
X_test_scaled= scaler.transform(X_test)
X_trained_scaled[0]

In [ ]:
print(Y_train.unique())

In [ ]:
#Different types of hidden layer
# acclist = []
for i in range(5, 129, 5):
    print(i)
#     model = Sequential()
#     model.add(Dense(128,activation='relu', input_shape=(X_trained_scaled.shape[1],) ))
#     model.add(Dense(i,activation='relu'))
#     model.add(Dense(2,activation='sigmoid'))
#     model.compile(loss='sparse_categorical_crossentropy',optimizer='Adam',metrics=['accuracy'])
#     history = model.fit(X_trained_scaled,Y_train_encoded,epochs=50)
#     y_prob = model.predict(X_test_scaled)
#     y_pred2 = y_prob.argmax(axis=1)
# #     print(f"Accuracy Score of ANN at {i}: ",accuracy_score(Y_test_encoded, y_pred2))
#     acclist.append(accuracy_score(Y_test_encoded, y_pred2))
# print(acclist)

In [ ]:
model = Sequential()

# model.add(Flatten(input_shape=(28,28)))
model.add(Dense(128,activation='relu', input_shape=(X_trained_scaled.shape[1],) ))
model.add(Dense(100,activation='relu'))
model.add(Dense(2,activation='sigmoid'))
model.summary()


In [ ]:
X_trained_scaled
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
Y_train_encoded= label_encoder.fit_transform(Y_train)
Y_test_encoded= label_encoder.fit_transform(Y_test)
Y_train_encoded, Y_test_encoded

In [ ]:
model.compile(loss='sparse_categorical_crossentropy',optimizer='Adam',metrics=['accuracy'])
history = model.fit(X_trained_scaled,Y_train_encoded,epochs=50)
y_prob = model.predict(X_test_scaled)

In [ ]:
y_prob

In [ ]:
y_pred2 = y_prob.argmax(axis=1)
print(y_pred2,"\n", Y_test_encoded, Y_test_encoded.shape)

In [ ]:
print("ConfusionMatrix of ANN: : \n",confusion_matrix(Y_test_encoded, y_pred2))
print("Accuracy Score of ANN: ",accuracy_score(Y_test_encoded, y_pred2))

In [ ]:
precision, recall, f1_score, support = precision_recall_fscore_support(Y_test_encoded, y_pred2)
print("Precision is : ", precision)
print("Recall is : ", recall)
print("f1 score is : ", f1_score)